Idea of this notebook: 

Time series prediction of daily new Covid-Cases in Chile, with CASEN data as features

Predict what?
* For each region or for the whole country, predict future development (next five or ten days) of Covid cases/deaths
* y = number of cases
* x = casen data + more features (s. time series tips) 

data: CASEN data
Covid data: from MINSAL


1. Read in data: CASEN data, data from MINSAL about regions
2. Data exploration
3. Data visualization
3. Metrics: low RSME (good for time series) 
4. Get features with high correlation
5. build model with those features (maybe LR)
6. GridSearch with different models (LR, Kregressor, RandomForestRegressor) 
7. Findings/conclusion


In [ ]:

# Read in data about Covid-19 in Chilean districts
cases_districts = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv')
cases_districts = cases_districts[~cases_districts.Comuna.str.contains('Desconocido')]


deaths_districts = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv')
deaths_districts = deaths_districts[~deaths_districts.Comuna.str.contains('Desconocido')]
deaths_districts = deaths_districts[~deaths_districts.Comuna.str.contains('Total')]

In [ ]:
def timeline_plot(df, title):
    """
    A function to plot a seaborn diagram which shows the development of Covid-19 data in Chilean Regions. 
    For better readability, the function returns a log scale plot. 
    
    Input: 
        df (DataFrame): A DataFrame whith information about Covid-19 in Chile.
        title (string): Specify the kind of the plot (cases or deaths)
        
    Output: 
        A seaborn plot. 
    """

    # Get only relevant columns with information about cases/deaths:     
    df_numbers = df[df.columns[5:-1]]
    df_final = pd.concat([df['Region'], df_numbers], axis = 1)

    # Group by region to get total numbers for each region. 
    df_grouped = df_final.groupby('Region')[[i for i in df_final.columns[1:]]].sum().reset_index()

    # Apply melt, convert Date-column to datetime and sort values
    df_melt = pd.melt(df_grouped, id_vars = 'Region', 
                       value_vars = df_grouped.columns.drop('Region'),
                       var_name='Date', 
                       value_name='Cases')

    df_melt.loc[:, ['Date']] = pd.to_datetime(df_melt['Date'])
    df_melt = df_melt.sort_values('Cases', ascending = False)

    # Plot the timeline
    f, ax = plt.subplots(figsize = (16, 12))

    g = sns.lineplot(
        df_melt.Date, 
        df_melt.Cases, 
        hue = df_melt.Region
    )

    plt.xlabel('Date', fontsize = 20)
    plt.ylabel('{} (log scale)'.format(title), fontsize = 20)
    plt.title('Covid-19 {} in Chile (log scale)'.format(title), fontsize = 25)

    ax.yaxis.tick_right()
    ax.set_yscale('log')
    plt.tick_params(labelsize=20, rotation=90)
    plt.legend(scatterpoints=1, frameon=True, labelspacing=.2, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.5)

    plt.grid(False)
    ax.yaxis.grid()
    sns.despine()
    plt.show()

In [ ]:
    #(np.sum(casen.isna() == True)/casen.shape[0]).sort_values(ascending = False).head(50)

In [ ]:
mobility_chile_target = mobility_chile[mobility_chile['sub_region_1'] == target]

merged_mobility = df_sel.merge(mobility_chile_target, left_on = 'Region', right_on = 'date', how = 'left')
merged_mobility = merged_mobility.drop(['sub_region_1', 'date'], axis = 1)
merged_mobility = merged_mobility[~merged_mobility['retail_and_recreation_percent_change_from_baseline'].isna()]

merged_mobility

In [ ]:
transposed_cases.head(2)

In [ ]:
mean_error = []
for day in range(3, 157):
    train = df_mobility[df_mobility['days'] < day]
    val = df_mobility[df_mobility['days'] == day]

    p = val['yesterday_value'].values

    error = rmsle(val['cases'].values, p)
    print('day %d - Error %.5f' % (day, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

In [ ]:
#    mobility_df = prepare_mobility_data(df, 'Germany', '2020-04-04', '2020-08-07', include_regions=False, drop_date=False)
 #       sns.lineplot(x = mobility_chile_all['date'], y = mobility_chile_all['grocery_and_pharmacy_percent_change_from_baseline'])
  #      sns.lineplot(x = mobility_chile_all['date'], y = mobility_chile_all['transit_stations_percent_change_from_baseline'])
   #     sns.lineplot(x = mobility_chile_all['date'], y = mobility_chile_all['parks_percent_change_from_baseline'])
    #    sns.lineplot(x = mobility_chile_all['date'], y = mobility_chile_all['workplaces_percent_change_from_baseline'])
     #   sns.lineplot(x = mobility_chile_all['date'], y = mobility_chile_all['residential_percent_change_from_baseline'])

In [ ]:
df['sub_region_1'] = df['sub_region_1'].map({
                                                'Tarapacá': 'Tarapacá',
                                                'Antofagasta': 'Antofagasta',
                                                'Atacama': 'Atacama',
                                                'Coquimbo': 'Coquimbo',
                                                'Valparaíso': 'Valparaíso',
                                                "O'Higgins": 'O’Higgins',
                                                'Maule': 'Maule',
                                                'Bio Bio': 'Biobío',
                                                'Araucania': 'Araucanía', 
                                                'Los Lagos': 'Los Lagos',
                                                'Aysén': 'Aysén',
                                                'Magallanes and Chilean Antarctica': 'Magallanes',
                                                'Santiago Metropolitan Region': 'Metropolitana',
                                                'Los Ríos': 'Los Ríos',
                                                'Arica y Parinacota': 'Arica y Parinacota',
                                                'Ñuble': 'Ñuble'
                                            })

In [ ]:


mobility['sub_region_1'] = mobility['sub_region_1'].map({
    'Tarapacá': 'Tarapacá',
    'Antofagasta': 'Antofagasta',
    'Atacama': 'Atacama',
    'Coquimbo': 'Coquimbo',
    'Valparaíso': 'Valparaíso',
    "O'Higgins": 'O’Higgins',
    'Maule': 'Maule',
    'Bio Bio': 'Biobío',
    'Araucania': 'Araucanía', 
    'Los Lagos': 'Los Lagos',
    'Aysén': 'Aysén',
    'Magallanes and Chilean Antarctica': 'Magallanes',
    'Santiago Metropolitan Region': 'Metropolitana',
    'Los Ríos': 'Los Ríos',
    'Arica y Parinacota': 'Arica y Parinacota',
    'Ñuble': 'Ñuble'
})

mobility_chile = mobility[(mobility['country_region'] == 'Chile') & (~mobility['sub_region_1'].isna())]

mobility_chile = mobility_chile.drop([
    'country_region',
    'country_region_code', 
    'sub_region_2', 
    'metro_area', 
    'iso_3166_2_code', 
    'census_fips_code'
], axis = 1)

mobility_chile.loc[:, 'date'] = pd.to_datetime(mobility_chile.loc[:, 'date'])

# Convert mobility columns to float
info_columns = ['retail_and_recreation_percent_change_from_baseline', 
                'grocery_and_pharmacy_percent_change_from_baseline',
               'parks_percent_change_from_baseline',
               'transit_stations_percent_change_from_baseline',
               'workplaces_percent_change_from_baseline',
               'residential_percent_change_from_baseline']

mobility_chile[info_columns] = mobility_chile[info_columns].astype(float)

mobility_chile = mobility_chile.groupby(['sub_region_1', 'date'])[info_columns].mean().reset_index()

mobility_chile = mobility_chile[(mobility_chile['date'] >= '2020-03-04') & (mobility_chile['date'] <= '2020-08-07')]

mobility_chile['days'] = 0

mobility_chile = mobility_chile.assign(days = np.arange(len(mobility_chile)) % 157)

mobility_chile = mobility_chile.drop(['date'], axis = 1)

mobility_chile

In [ ]:
# Define rsmle

def rmsle(ytrue, ypred):    
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [ ]:
df = cases_regions.copy()
title = 'cases'



df = df[(df['Region'] >= '2020-03-04') & (df['Region'] <= '2020-08-07')]

df.insert(0, 'days', range(len(df)))

df = df.drop(['Region'], axis = 1)


df_melt = pd.melt(
    df, id_vars = ['days'], 
    value_vars = df.columns.drop(['days']), 
    value_name=title
)

df_melt = df_melt.loc[df_melt['variable'] != 'Total', :]

df_melt['yesterday_value'] = df_melt.groupby('variable')[title].shift()

df_melt['yesterday_diff'] = df_melt.groupby('variable')['yesterday_value'].diff()

df_mobility = df_melt.merge(mobility_chile, left_on = ['days', 'variable'], right_on = ['days', 'sub_region_1'], how = 'left')

df_mobility = df_mobility.drop(['sub_region_1'], axis = 1)

df_mobility = df_mobility.fillna(method = 'bfill', axis = 0)

df_mobility.head()

In [ ]:
# Merge mobility and cases data with CASEN social indicators
df_mobility_casen = df_mobility.merge(casen_regions, left_on = 'variable', right_on = 'region', how = 'left')

# Make dummy columns for regions: 
region_dummies = pd.get_dummies(df_mobility_casen['variable'])

df_mobility_casen_dummies = pd.concat([df_mobility_casen, region_dummies], axis = 1)

df_mobility_casen_dummies = df_mobility_casen_dummies.drop(['variable', 'region'], axis = 1)

df_mobility_casen_dummies

In [ ]:
# To make sure the model is worth using I like to set a baseline score that it has to beat. In this case, 
# a reasonably strong baseline is using the last week amount of sales as a prediction for the sales this week.
# Baseline

mean_error = []
for day in range(3, 157):
    train = df_mobility_casen_dummies[df_mobility_casen_dummies['days'] < day]
    val = df_mobility_casen_dummies[df_mobility_casen_dummies['days'] == day]

    p = val['yesterday_value'].values
    
    error = rmsle(val['yesterday_diff'].values, p)
    print('day %d - Error %.5f' % (day, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

In [ ]:
y = df_mobility_casen_dummies['yesterday_diff']
X = df_mobility_casen_dummies.drop(['yesterday_diff'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
mean_error = []
for day in range(3, 157):

    mdl = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0)
    mdl.fit(X_train, y_train)

    y_pred = mdl.predict(X_test)

    error = rmsle(y_test, y_pred)
    print('day %d - Error %.5f' % (day, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))



In [ ]:
def rmse(actual, predict):
    """
    Function 
    """
    
    predict = np.array(predict)
    actual = np.array(actual)
    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    return score

rmse_score = make_scorer(rmse, greater_is_better = False)

In [ ]:
model = RandomForestRegressor()

param_search = { 
    'n_estimators': [20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(5,15)]
}

model = KNeighborsRegressor()

param_search = { 
    'n_neighbors': [5, 7, 10, 13, 20, 30, 50],
    'weights': ['uniform', 'distance'],
    'leaf_size': [10, 30, 50]
}

In [ ]:
# Prepare train and test set: Training set from March to June (4 months), validation July (1 month)
X_train = df_melt_mobility_num[:'2020-06-30'].drop(['new_cases'], axis = 1)
y_train = df_melt_mobility_num.loc[:'2020-06-30', 'new_cases']
X_test = df_melt_mobility_num['2020-07-01':].drop(['new_cases'], axis = 1)
y_test = df_melt_mobility_num.loc['2020-07-01':, 'new_cases']

y_train

In [ ]:
# Tryout of some models (https://towardsdatascience.com/time-series-modeling-using-scikit-pandas-and-numpy-682e3b8db8d1)

# Store models in list
models = []
models.append(('LR', LinearRegression()))
models.append(('KNN', KNeighborsRegressor())) 
models.append(('RF', RandomForestRegressor(n_estimators = 100, random_state = 42)))
models.append(('SVR', SVR(gamma='auto')))

results = []
names = []
for name, model in models:

    # TimeSeries split
    tscv = TimeSeriesSplit(n_splits=12)

    # TimeSeries Cross validation
    cv_results = cross_val_score(model, X_train, y_train, cv=tscv, scoring='r2')
    results.append(cv_results)
    names.append(name)
    
    # Print results
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    # Plot boxplot for each model
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

In [ ]:

model = KNeighborsRegressor()

param_search = { 
    'n_neighbors': [5, 7, 10, 13, 20, 30, 50],
    'weights': ['uniform', 'distance'],
    'leaf_size': [10, 30, 50]
}


tscv = TimeSeriesSplit(n_splits=12)

gsearch = GridSearchCV(
    estimator=model, 
    cv=tscv, 
    param_grid=param_search, 
    scoring = rmse_score)

gsearch.fit(X_train, y_train)

best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

y_true = y_test.values
y_pred = best_model.predict(X_test)

print(best_model)

regression_metrics(y_true, y_pred)

In [ ]:
imp = best_model.feature_importances_[:10]
features = X_train.columns

indices = np.argsort(imp)
plt.title('Feature Importances')
plt.barh(range(len(indices)), imp[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
fix, ax = plt.subplots(figsize = (12,8))
plt.scatter(y_true, y_pred)

In [ ]:
model = RandomForestRegressor()

param_search = { 
    'n_estimators': [20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(5,15)]
}

tscv = TimeSeriesSplit(n_splits=12)

gsearch = GridSearchCV(
    estimator=model, 
    cv=tscv, 
    param_grid=param_search, 
    scoring = rmse_score)

gsearch.fit(X_train_CASEN, y_train_CASEN)

best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

y_true_CASEN = y_test_CASEN.values
y_pred_CASEN = best_model.predict(X_test_CASEN)

print(best_model)

regression_metrics(y_true_CASEN, y_pred_CASEN)

In [ ]:
fix, ax = plt.subplots(figsize = (16,10))
plt.scatter(y_true_CASEN, y_pred_CASEN)

In [ ]:
imp = best_model.feature_importances_[:10]
features = X_train.columns

indices = np.argsort(imp)
plt.title('Feature Importances')
plt.barh(range(len(indices)), imp[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# Prepare train and test set: Training set from March to June (4 months), validation July (1 months)
X_train = df_melt_mobility_num[:'2020-06-30'].drop(['new_deaths'], axis = 1)
y_train = df_melt_mobility_num.loc[:'2020-06-30', 'new_deaths']
X_test = df_melt_mobility_num['2020-07-01':].drop(['new_deaths'], axis = 1)
y_test = df_melt_mobility_num.loc['2020-07-01':, 'new_deaths']

In [ ]:
# Store models in list
models = []
models.append(('LR', LinearRegression()))
models.append(('KNN', KNeighborsRegressor())) 
models.append(('RF', RandomForestRegressor(n_estimators = 100, random_state = 42)))
models.append(('SVR', SVR(gamma='auto')))

results = []
names = []
for name, model in models:

    # TimeSeries split, n_splits = 6
    tscv = TimeSeriesSplit(n_splits=12)

    # TimeSeries Cross validation
    cv_results = cross_val_score(model, X_train, y_train, cv=tscv, scoring='r2')
    results.append(cv_results)
    names.append(name)
    
    # Print results
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    # Plot boxplot for each model
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

In [ ]:
model = RandomForestRegressor()

param_search = { 
    'n_estimators': [20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(5,15)]
}


tscv = TimeSeriesSplit(n_splits=12)

gsearch = GridSearchCV(
    estimator=model, 
    cv=tscv, 
    param_grid=param_search, 
    scoring = rmse_score)

gsearch.fit(X_train, y_train)

best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

y_true = y_test.values
y_pred = best_model.predict(X_test)

print(best_model)

regression_metrics(y_true, y_pred)

In [ ]:
imp = best_model.feature_importances_[:10]
features = X_train.columns

indices = np.argsort(imp)
plt.title('Feature Importances')
plt.barh(range(len(indices)), imp[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
model = RandomForestRegressor()

param_search = { 
    'n_estimators': [20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(5,15)]
}

tscv = TimeSeriesSplit(n_splits=12)

gsearch = GridSearchCV(
    estimator=model, 
    cv=tscv, 
    param_grid=param_search, 
    scoring = rmse_score)

gsearch.fit(X_train_CASEN, y_train_CASEN)

best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

y_true_CASEN = y_test_CASEN.values
y_pred_CASEN = best_model.predict(X_test_CASEN)

print(best_model)

regression_metrics(y_true_CASEN, y_pred_CASEN)

In [ ]:
imp = best_model.feature_importances_[:10]
features = X_train.columns

indices = np.argsort(imp)
plt.title('Feature Importances')
plt.barh(range(len(indices)), imp[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()